In [2]:
import os
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

In [3]:
from bs4 import BeautifulSoup
from collections import defaultdict
from flow_wmd.documents import Document
from flow_wmd.models import LC_RWMD, WMD, WMDManyToMany
from gensim.models import KeyedVectors
from itertools import islice
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cluster

import numpy as np
import pandas as pd
import re

%load_ext autoreload
%autoreload 2

/Users/mikaelbrunila/Documents/Code/flow-wmd/venv-fwmd/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## 1. Prepare IMDB data

### 1.1 Load data and stopwords.

In [4]:
PATH = "../data/"
imdb_data = pd.read_csv(f"{PATH}IMDB_Dataset.csv")
stopword_list=stopwords.words('english')

### 1.2 Initialize cleanup functions 

Functions to remove noisy formatting, lemmatizing and removing stopwords.

In [5]:
# Custom preprocessing functions
# Partly self-authored, partly from https://www.kaggle.com/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews

#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square <brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = re.sub('<br / ><br / >', ' ', text)
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z\s]'
    text=re.sub(pattern,'',text)
    return text

#Lemmatizing the text
def simple_lemmatizer(text):
    lemmatizer=WordNetLemmatizer() 
    text= ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

#removing the stopwords
def remove_stopwords(text, stopword_list, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token.lower() for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

### 1.3 Remove special formatting and stopwords

Remove stopwords before denoising, lemmatizing and removing special characters.

In [6]:
%time 

tokenizer=ToktokTokenizer()
imdb_data['review_clean']= [remove_stopwords(r, stopword_list) for r in imdb_data['review']]

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.05 µs


Denoise, remove special characters, lemmatize.

In [7]:
%time

imdb_data['review_clean']=imdb_data['review_clean'].apply(denoise_text)
imdb_data['review_clean']=imdb_data['review_clean'].apply(remove_special_characters)
imdb_data['review_clean']=imdb_data['review_clean'].apply(simple_lemmatizer)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


Remove stopwords again, after other preprocessing.

In [8]:
%time 

imdb_data['review_clean']= [remove_stopwords(r, stopword_list) for r in imdb_data['review_clean']]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


Data _before_ preprocessing.

In [9]:
imdb_data['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

Data _after_ preprocessing.

In [10]:
imdb_data['review_clean'][0]

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz m

### 1.4 Separate pos and neg reviews

In [162]:
pos = imdb_data[imdb_data.sentiment == "positive"].reset_index(drop=True)
neg = imdb_data[imdb_data.sentiment == "negative"].reset_index(drop=True)

In [163]:
pos = pos.review_clean.tolist()
neg = neg.review_clean.tolist()

## 2. WMD

### 2.1 Tokenize and "sample" data

In [164]:
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    return tokens

pos_tok = list(map(tokenize, pos[:500]))
neg_tok = list(map(tokenize, neg[:500]))

In [165]:
pos_sample = [" ".join(doc) for doc in pos_tok]
neg_sample = [" ".join(doc) for doc in neg_tok]

### 2.2 Load pretrained Google News W2V model

In [166]:
def read_1w_corpus(name, sep="\t"):
    for line in open(name):
        yield line.split(sep)

print("Loading GoogleNews Vectors")
%time model = KeyedVectors.load_word2vec_format('../embeddings/GoogleNews-vectors-negative300.bin.gz', binary=True)

Loading GoogleNews Vectors
CPU times: user 47.8 s, sys: 4.21 s, total: 52.1 s
Wall time: 53.8 s


### 2.3 Load corpus and remove OOV words

In [167]:
corpus = pos_sample + neg_sample

%time vectorizer = TfidfVectorizer(use_idf=False, tokenizer=tokenize, norm='l1')
%time vectorizer.fit(corpus)

CPU times: user 99 µs, sys: 820 µs, total: 919 µs
Wall time: 1.51 ms


/Users/mikaelbrunila/Documents/Code/flow-wmd/venv-fwmd/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CPU times: user 363 ms, sys: 1.06 s, total: 1.42 s
Wall time: 2.87 s


TfidfVectorizer(norm='l1', tokenizer=<function tokenize at 0x7fadd27fb280>,
                use_idf=False)

In [168]:
%time oov = [word for word in vectorizer.get_feature_names() if word not in model.key_to_index.keys()]

CPU times: user 24.8 ms, sys: 5.47 ms, total: 30.3 ms
Wall time: 35.4 ms


In [169]:
len(oov)

3066

In [170]:
#removing the oov words
def remove_oov(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in oov]
    #filtered_tokens = filter(lambda token: token not in oov, tokens)
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

%time pos_sample = list(map(remove_oov, pos_sample))
%time neg_sample = list(map(remove_oov, neg_sample))

CPU times: user 2.49 s, sys: 23.2 ms, total: 2.51 s
Wall time: 2.54 s
CPU times: user 2.33 s, sys: 13.4 ms, total: 2.34 s
Wall time: 2.35 s


In [171]:
pos_sample[0]

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given maximum security state focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz may become comfortab

In [172]:
corpus = pos_sample + neg_sample

%time vectorizer = TfidfVectorizer(use_idf=True, tokenizer=tokenize,norm='l1')
%time vectorizer.fit(corpus)

CPU times: user 29 µs, sys: 1 µs, total: 30 µs
Wall time: 31.9 µs
CPU times: user 223 ms, sys: 9.34 ms, total: 232 ms
Wall time: 242 ms


TfidfVectorizer(norm='l1', tokenizer=<function tokenize at 0x7fadd27fb280>)

Bag-of-words vectorizer.

In [173]:
%time
pos_nbow = vectorizer.transform(pos_sample)
neg_nbow = vectorizer.transform(neg_sample)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


In [174]:
pos_tok = list(map(tokenize, pos_sample))
neg_tok =  list(map(tokenize, neg_sample))

In [175]:
pos_tok[0][:20]

['one',
 'reviewer',
 'mentioned',
 'watching',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scene',
 'violence',
 'set',
 'right']

In [176]:
%time oov_ = [word for word in vectorizer.get_feature_names() if word not in model.key_to_index.keys()]

CPU times: user 20.2 ms, sys: 654 µs, total: 20.8 ms
Wall time: 20.7 ms


In [177]:
len(oov_)

0

### 2.4 Get features and embeddings

In [178]:
features = vectorizer.get_feature_names()
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
idx2word = {idx: word for idx, word in enumerate(vectorizer.get_feature_names())}

Get the embedding matrix "E" for all features.

In [179]:
E = np.vstack([model.get_vector(word) for word in vectorizer.get_feature_names()])

### Cluster

In [97]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [231]:
X = model[features]

In [234]:
NUM_CLUSTERS = 100
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS, verbose=0)
kmeans.fit(X)

KMeans(n_clusters=100)

In [235]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [237]:
word2cluster = {features[idx]: cl for idx, cl in enumerate(labels)}

In [238]:
print(take(10, word2cluster.items()))

[('`', 60), ('aaargh', 60), ('aamir', 60), ('aaron', 81), ('ab', 1), ('abandon', 10), ('abandoned', 0), ('abba', 60), ('abbey', 68), ('abbot', 18)]


In [239]:
cluster2words = defaultdict(list)
for key, value in word2cluster.items():
    cluster2words[value].append(key)

In [240]:
print(cluster2words[0][:100])

['abandoned', 'abused', 'accosted', 'alive', 'amputated', 'asleep', 'assassinated', 'attacked', 'avenge', 'beaten', 'beheading', 'belonging', 'bitten', 'blinded', 'blindfolded', 'bloodstained', 'bombed', 'brutally', 'buried', 'burned', 'burnt', 'butchered', 'castrated', 'chained', 'chased', 'confronted', 'crushed', 'cursed', 'dead', 'deposed', 'desecrated', 'die', 'died', 'disarmed', 'disfigured', 'drowned', 'dumped', 'fatally', 'fired', 'fled', 'flee', 'fleeing', 'flogged', 'foiled', 'freed', 'grazed', 'gunned', 'gunning', 'hacked', 'hanged', 'harassed', 'harassing', 'hazed', 'hijacked', 'holed', 'hospitalized', 'hostage', 'hunted', 'infected', 'injured', 'kidnapped', 'kill', 'killed', 'killing', 'lynch', 'manhandled', 'menaced', 'misbehaving', 'missing', 'murdered', 'mutilate', 'mutilated', 'overthrown', 'poisoned', 'punished', 'raided', 'rampaged', 'raped', 'repelled', 'rescued', 'revenge', 'rob', 'robbed', 'semiconscious', 'severed', 'slain', 'slept', 'slit', 'stabbed', 'starving',

### 2.5 Initialize documents

Transform all reviews into "documents", each with a set of weights per word in the corpus ("nbow"), the sum of these weights ("weights_sum"), the indeces of the words in the documents ("idxs") and the word vectors corresponding to each word ("vecs").

In [241]:
%time 

pos_docs, neg_docs = [], []

for idx, doc in enumerate(pos_tok):
    pos_docs.append(Document(doc, pos_nbow[idx], word2idx, E))
    
for idx, doc in enumerate(neg_tok):
    neg_docs.append(Document(doc, neg_nbow[idx], word2idx, E))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [242]:
pos_docs[0].nbow

array([[0., 0., 0., ..., 0., 0., 0.]])

In [243]:
pos_docs[0].weights_sum

1.0

In [244]:
pos_docs[0].idxs[:10]

[12299, 9740, 1036, 2071, 10266, 5148, 13342, 13344, 11302, 5673]

In [245]:
pos_docs[0].vecs[:1][0][:10]

array([ 0.16992188,  0.04907227,  0.08154297,  0.12011719, -0.14746094,
        0.0291748 ,  0.36523438, -0.10107422,  0.125     ,  0.04516602],
      dtype=float32)

### 2.6 Linear-Complexity Relaxed WMD (LC-RWMD)

Run the [Linear-Complexity Relaxed WMD](https://arxiv.org/abs/1711.07227) to get the distances between all positive and all negative reviews.

In [246]:
%time lc_rwmd = LC_RWMD(pos_docs, neg_docs,pos_nbow,neg_nbow,E)
%time lc_rwmd.get_D()
#%time lc_rwmd.get_L(1)
#%time lc_rwmd.get_rwmd()

CPU times: user 606 µs, sys: 1.03 ms, total: 1.63 ms
Wall time: 4.21 ms
CPU times: user 2min 15s, sys: 22.3 s, total: 2min 37s
Wall time: 41.4 s


### 2.7 Gale-Shapeley Pairing

Use the [Gale-Shapeley matching algorithm](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) to find the optimal pairs between positive and negative reviews. This iterates over all the reviews and finds the set of matches that pairs each review with its optimal match given that all positive reviews have to be matched with a negative review and vice versa. The output is a dictionary of key-value pairs, where each pair represents an optimal match.

In [247]:
from flow_wmd.gale_shapeley import Matcher

matcher = Matcher(lc_rwmd.D)
engaged = matcher.matchmaker()
matcher.check()
pairs = engaged

Let's look at the output of Gale-Shapeley:

In [248]:
take(10, pairs.items())

[(137, 497),
 (388, 468),
 (95, 360),
 (454, 361),
 (259, 363),
 (263, 146),
 (143, 90),
 (224, 107),
 (91, 108),
 (443, 347)]

### 2.8 Pairwise WMD

Calculate the pairwise distances between the documents selected by the Galey-Shapeley algorithm _without_ returning the flow between individual words.

In [137]:
from flow_wmd.models import WMDPairs

wmd_pairs = WMDPairs(pos_docs,neg_docs,pairs,E,idx2word)
%time wmd_pairs.get_distances()

Calculated distances between 0 documents.
Calculated distances between 100 documents.
Calculated distances between 200 documents.
Calculated distances between 300 documents.
Calculated distances between 400 documents.
CPU times: user 4min 10s, sys: 16.1 s, total: 4min 26s
Wall time: 2min 58s


The return value is a matrix of distances between the document pairs.

In [193]:
wmd_pairs.distances

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Calculate the pairwise distances between the documents selected by the Galey-Shapeley algorithm, this time also returning the flow between individual words.

In [350]:
wmd_pairs_flow = WMDPairs(pos_docs,neg_docs,pairs,E,idx2word)
%time wmd_pairs_flow.get_distances(return_flow = True, sum_clusters = True, w2c = word2cluster, c2w = cluster2words)

Calculated distances between 0 documents.
Calculated distances between 100 documents.
Calculated distances between 200 documents.
Calculated distances between 300 documents.
Calculated distances between 400 documents.
CPU times: user 4min 40s, sys: 17.2 s, total: 4min 57s
Wall time: 3min 25s


Now we have three return values.

The first one is again a matrix of distances between the document pairs.

In [351]:
wmd_pairs_flow.distances

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

The second return value is a list of tuples with all the words that contributed the most to the distance from the positive documents to the negative ones. These are _not_ sorted from high to low or vice versa.

In [352]:
take(10, wmd_pairs_flow.wc_X1.items())

[('exploration', 0.049010000000000005),
 ('double', 0.18775),
 ('delinquency', 0.20461),
 ('distortion', 0.11278),
 ('pet', 0.24564),
 ('granddaughter', 0.1211),
 ('hubris', 0.03926),
 ('hardened', 0.04453),
 ('sleeker', 0.03),
 ('pu', 0.02374)]

The third return value is a list of tuples with all the words that contributed the most to the distance from the negative documents to the positive ones. Again, these are _not_ sorted from high to low or vice versa.

In [353]:
take(10, wmd_pairs_flow.wc_X2.items())

[('double', 0.12838),
 ('pet', 0.16274000000000002),
 ('hardened', 0.07715),
 ('lurch', 0.0698),
 ('underwear', 0.02577),
 ('corpse', 0.026479999999999997),
 ('primeval', 0.043500000000000004),
 ('threesome', 0.10307999999999999),
 ('skate', 0.15538),
 ('ritualistic', 0.06027)]

In [354]:
take(10, wmd_pairs_flow.cc_X1.items())

[(60, 23.696899999999985),
 (81, 39.47520000000002),
 (1, 16.666900000000005),
 (10, 37.608569999999965),
 (0, 7.430600000000002),
 (68, 6.140189999999999),
 (18, 8.9413),
 (90, 21.92162),
 (92, 8.460380000000004),
 (19, 6.029949999999999)]

In [355]:
take(10, wmd_pairs_flow.cc_X2.items())

[(60, 17.831660000000003),
 (81, 22.719140000000017),
 (1, 15.736809999999993),
 (10, 35.36103999999994),
 (0, 11.203750000000003),
 (68, 3.5156499999999995),
 (18, 10.151929999999995),
 (90, 24.523450000000004),
 (92, 10.510490000000006),
 (19, 7.159270000000002)]

In [356]:
{k: v for k, v in sorted(wmd_pairs_flow.cc_X1.items(), key=lambda item: item[1], reverse=True)[:10]}

{36: 89.03332999999994,
 62: 84.44415999999983,
 48: 44.972770000000025,
 81: 39.47520000000002,
 10: 37.608569999999965,
 27: 32.46946,
 52: 32.33321000000001,
 58: 32.24520000000002,
 75: 31.084949999999996,
 25: 30.576999999999995}

### 2.9 Intepreting pairwise WMD flows

Now, let's sort the distances of the words that created the most distance from the positive to the negative reviews.

In [357]:
{k: v for k, v in sorted(wmd_pairs_flow.wc_X1.items(), key=lambda item: item[1], reverse=True)[:30]}

{'film': 4.534129999999999,
 'movie': 3.938469999999998,
 'story': 3.434129999999999,
 'great': 3.077940000000002,
 'love': 2.9721399999999996,
 'performance': 2.73945,
 'life': 2.426700000000001,
 'character': 2.39385,
 'best': 2.340230000000001,
 'well': 2.3356999999999988,
 'watch': 2.2191299999999994,
 'show': 2.1959499999999994,
 'still': 2.195579999999999,
 'scene': 2.19522,
 'good': 2.1382,
 'time': 2.09075,
 'comedy': 2.08384,
 'young': 2.0808300000000015,
 'one': 2.0784700000000003,
 'like': 2.015480000000001,
 'dvd': 2.01107,
 'excellent': 2.0090300000000005,
 'people': 1.9920699999999996,
 'fan': 1.9595799999999997,
 'actor': 1.87092,
 'role': 1.86876,
 'see': 1.8347,
 'little': 1.8331200000000005,
 'loved': 1.83208,
 'family': 1.7825199999999999}

Next, let's see what added most distance when moving from the negative to the positive reviews.

In [358]:
{k: v for k, v in sorted(wmd_pairs_flow.wc_X2.items(), key=lambda item: item[1], reverse=True)[:30]}

{'movie': 6.2731,
 'bad': 5.4052500000000006,
 'film': 4.1417399999999995,
 'worst': 3.635940000000001,
 'plot': 3.55549,
 'waste': 3.1293699999999993,
 'scene': 3.013090000000001,
 'acting': 2.7281000000000004,
 'character': 2.5816599999999994,
 'made': 2.4689400000000004,
 'rating': 2.45743,
 'like': 2.45132,
 'ever': 2.3756300000000006,
 'stupid': 2.34381,
 'actor': 2.2481300000000006,
 'story': 2.22648,
 'funny': 2.2264,
 'really': 2.2117,
 'boring': 2.2083199999999996,
 'watch': 2.19631,
 'good': 2.1373299999999995,
 'even': 2.0962,
 'thing': 2.0919399999999997,
 'money': 2.0855600000000005,
 'watching': 2.039080000000001,
 'guy': 1.9936299999999998,
 'awful': 1.9861900000000003,
 'would': 1.9547700000000006,
 'seen': 1.9501299999999995,
 'see': 1.93109}

In [359]:
wmd_pairs_flow.cc_X2.items()

dict_items([(60, 17.831660000000003), (81, 22.719140000000017), (1, 15.736809999999993), (10, 35.36103999999994), (0, 11.203750000000003), (68, 3.5156499999999995), (18, 10.151929999999995), (90, 24.523450000000004), (92, 10.510490000000006), (19, 7.159270000000002), (70, 19.557169999999992), (96, 1.8248), (34, 32.90991000000001), (47, 21.336429999999993), (40, 1.1524), (52, 22.593739999999997), (74, 10.246), (75, 24.458830000000013), (73, 5.03237), (48, 46.176009999999955), (3, 12.960470000000008), (62, 77.90586000000003), (43, 13.134669999999987), (4, 14.312549999999991), (36, 102.26358999999995), (87, 23.00473000000001), (32, 4.854150000000001), (56, 7.2088199999999985), (33, 23.04513), (54, 9.73582000000001), (61, 7.750219999999999), (24, 20.703079999999993), (9, 13.617820000000012), (80, 8.937149999999999), (12, 8.760590000000002), (23, 35.758649999999996), (85, 24.82207), (5, 8.449319999999998), (57, 21.36416), (6, 11.796859999999995), (8, 12.94871000000001), (31, 9.3619699999999

In [363]:
n_clusters = 50
n_words = 10

def output_clusters(wc, cc, c2w, n_clusters, n_words):
    top_clusters = [k for k, v in sorted(cc, key=lambda item: item[1], reverse=True)[:n_clusters]]
    print(take(10, cc))
    word_rankings = {k: v for k, v in sorted(wc, key=lambda item: item[1], reverse=True)}
    keywords = []
    for c in top_clusters:
        cluster_words = {w: word_rankings[w] for w in c2w[c] if w in word_rankings.keys()}
        top_c_words = [f"{k} ({round(v, 2)})" for k, v in sorted(cluster_words.items(), 
                                               key=lambda item: item[1], 
                                               reverse=True)[:n_words]]
        keywords.append(top_c_words)
    keywords_df = pd.DataFrame(keywords).transpose()
    keywords_df.columns = top_clusters
    return keywords_df

c1 = output_clusters(wmd_pairs_flow.wc_X1.items(), wmd_pairs_flow.cc_X1.items(), cluster2words, n_clusters, n_words)
c2 = output_clusters(wmd_pairs_flow.wc_X2.items(), wmd_pairs_flow.cc_X2.items(), cluster2words, n_clusters, n_words)

[(60, 23.696899999999985), (81, 39.47520000000002), (1, 16.666900000000005), (10, 37.608569999999965), (0, 7.430600000000002), (68, 6.140189999999999), (18, 8.9413), (90, 21.92162), (92, 8.460380000000004), (19, 6.029949999999999)]
[(60, 17.831660000000003), (81, 22.719140000000017), (1, 15.736809999999993), (10, 35.36103999999994), (0, 11.203750000000003), (68, 3.5156499999999995), (18, 10.151929999999995), (90, 24.523450000000004), (92, 10.510490000000006), (19, 7.159270000000002)]


In [364]:
c1

,36,62,48,81,10,27,52,58,75,25,...,78,65,11,9,61,63,39,32,49,43
0,well (2.34),performance (2.74),effect (1.15),john (1.06),make (1.57),great (3.08),u (1.35),role (1.87),man (1.67),dvd (2.01),...,fan (1.96),mickey (0.68),house (0.85),black (1.0),art (1.16),evil (0.93),heart (0.94),romantic (1.19),taste (0.54),question (0.87)
1,still (2.2),show (2.2),human (1.03),robert (0.91),get (1.57),excellent (2.01),oscar (0.78),work (1.72),old (1.65),tv (1.47),...,actor (1.87),guy (0.62),seat (0.81),white (0.75),school (0.88),mystery (0.76),brain (0.67),seductive (0.42),meatball (0.48),problem (0.66)
2,like (2.02),time (2.09),high (0.86),mary (0.89),take (1.54),wonderful (1.65),cant (0.51),part (1.38),friend (1.31),italian (1.32),...,hero (0.87),hell (0.56),room (0.55),brown (0.67),lesson (0.58),ghost (0.44),memory (0.67),artistic (0.4),popcorn (0.32),situation (0.66)
3,people (1.99),one (2.08),number (0.84),joe (0.84),find (1.36),entertaining (1.41),batman (0.41),relationship (0.98),girl (1.22),disney (1.19),...,writer (0.68),stuff (0.54),street (0.51),accent (0.59),student (0.45),universe (0.4),gene (0.61),imagery (0.36),ham (0.3),reason (0.56)
4,little (1.83),year (1.57),small (0.82),tom (0.81),go (1.35),perfect (1.36),betty (0.41),theme (0.95),actress (1.1),american (1.18),...,artist (0.44),yes (0.5),glass (0.5),costume (0.52),chemistry (0.44),zodiac (0.36),hospital (0.55),subtle (0.33),eat (0.26),flaw (0.52)
5,real (1.76),cast (1.51),usual (0.77),james (0.77),keep (1.3),brilliant (1.27),fx (0.41),direction (0.8),father (1.06),hollywood (1.09),...,guest (0.34),crazy (0.47),wheelchair (0.5),jean (0.5),academy (0.44),planet (0.35),disease (0.36),visually (0.32),fat (0.26),issue (0.49)
6,much (1.72),first (1.32),major (0.76),jim (0.77),give (1.28),nice (1.14),fav (0.41),experience (0.77),woman (1.04),york (0.7),...,reviewer (0.33),okay (0.39),box (0.4),hair (0.37),class (0.44),villain (0.32),psychological (0.34),mannerism (0.3),cream (0.26),challenge (0.39)
7,way (1.57),special (1.22),across (0.74),george (0.76),become (0.94),amazing (1.08),reese (0.35),atmosphere (0.75),brother (0.99),australia (0.6),...,leader (0.3),momma (0.39),bar (0.32),makeup (0.34),photography (0.4),witch (0.31),facial (0.27),visual (0.29),oz (0.25),lack (0.35)
8,many (1.54),day (1.18),typical (0.7),jones (0.75),turn (0.88),interesting (1.0),pb (0.33),depth (0.7),kid (0.99),america (0.58),...,critic (0.29),hey (0.32),apartment (0.31),blue (0.34),computer (0.35),creepy (0.3),blood (0.25),wistful (0.29),eating (0.25),difficulty (0.3)
9,really (1.5),short (1.15),rating (0.69),mr (0.74),meet (0.87),awesome (0.94),victoria (0.32),future (0.66),mother (0.89),japan (0.58),...,performer (0.24),oh (0.29),hall (0.26),color (0.24),college (0.34),pirate (0.29),ill (0.24),symbolism (0.23),tucker (0.21),difference (0.29)


In [365]:
c2 

,36,62,48,23,10,34,41,86,26,58,...,98,30,43,64,3,8,78,76,6,13
0,like (2.45),one (1.93),rating (2.46),watch (2.2),get (1.88),would (1.95),movie (6.27),got (1.58),action (1.78),idea (1.34),...,pretty (1.61),bored (1.11),lack (1.29),sense (1.32),life (1.39),name (1.28),actor (2.25),animal (0.88),poorly (1.12),worse (1.22)
1,ever (2.38),show (1.92),effect (1.3),see (1.93),make (1.87),could (1.78),film (4.14),saw (1.09),comment (1.32),work (1.24),...,totally (1.06),sorry (0.87),problem (1.21),talent (1.14),child (1.23),star (1.07),fan (1.25),creature (0.79),badly (0.75),slow (0.87)
2,really (2.21),time (1.74),main (0.87),say (1.86),go (1.76),try (1.28),scene (3.01),seemed (1.06),case (1.29),part (1.09),...,completely (0.95),tired (0.79),reason (1.17),charm (0.72),care (0.9),world (0.8),writer (0.98),fox (0.65),relentlessly (0.69),serious (0.77)
3,even (2.1),cast (1.62),low (0.86),look (1.64),find (1.36),want (1.26),viewer (1.15),played (0.96),review (1.21),direction (0.9),...,quite (0.82),confused (0.62),mess (0.73),imagination (0.43),family (0.79),blockbuster (0.58),producer (0.79),chick (0.56),quickly (0.61),hot (0.66)
4,thing (2.09),first (1.59),overall (0.84),recommend (1.15),save (1.17),enough (1.26),gore (1.1),gave (0.95),dialogue (1.16),premise (0.88),...,truly (0.82),hated (0.6),disappointment (0.56),emotion (0.29),live (0.49),dream (0.42),hero (0.72),rat (0.46),often (0.35),tedious (0.58)
5,much (1.77),end (1.57),standard (0.66),know (1.14),give (1.12),must (1.16),flick (1.06),went (0.93),budget (0.99),role (0.84),...,especially (0.46),sick (0.57),situation (0.49),oddness (0.27),kibbutz (0.45),hype (0.36),critic (0.56),spider (0.4),self (0.28),confusing (0.5)
6,people (1.74),least (1.5),single (0.66),mean (1.05),take (0.97),please (1.11),screen (0.91),watched (0.92),opinion (0.69),example (0.71),...,extremely (0.46),spoiled (0.43),trouble (0.46),sheer (0.27),native (0.37),former (0.36),godfather (0.39),hunter (0.38),ambiguously (0.23),impossible (0.45)
7,nothing (1.62),hour (1.48),along (0.64),understand (1.0),turn (0.91),trying (1.06),imdb (0.86),left (0.86),group (0.67),effort (0.65),...,highly (0.38),angry (0.42),difference (0.42),spirit (0.27),country (0.36),attention (0.34),reviewer (0.37),joey (0.37),briefly (0.22),extreme (0.39)
8,think (1.59),back (1.3),light (0.62),sit (0.99),keep (0.78),need (1.02),sequel (0.81),lost (0.84),act (0.66),view (0.61),...,generally (0.33),scared (0.41),question (0.39),ego (0.23),living (0.31),favorite (0.32),creator (0.36),whale (0.37),loosely (0.22),nasty (0.37)
9,lot (1.51),series (1.22),human (0.6),tell (0.96),use (0.76),looking (0.98),cinematography (0.6),came (0.81),call (0.57),belief (0.47),...,somewhat (0.32),wondering (0.39),mistake (0.36),brevity (0.23),justice (0.27),history (0.32),carver (0.23),reptile (0.33),randomly (0.21),tough (0.36)


## Appendix: Many-to-many WMD

This was a first attempt to do the flows from words between many documents, without first filtering using Gale-Shapeley. However, this proved too inefficient. As you can see looking at the CPU times, it is very slow even with extremely small samples and the time complexity is quadratic (or worse?), meaning it rapidly gets even worse as the sample size increases.

In [155]:
%time m2m_distances = WMDManyToMany(pos_docs[:20], neg_docs[:20],E,idx2word).get_distances(return_flow = False)

CPU times: user 1min 48s, sys: 12.7 s, total: 2min 1s
Wall time: 50 s


In [156]:
%time m2m_distances_flow, wc_X1, wc_X2 = WMDManyToMany(pos_docs[:20],neg_docs[:20],E,idx2word).get_distances(return_flow = True)

CPU times: user 1min 52s, sys: 13.5 s, total: 2min 5s
Wall time: 51.4 s


In [157]:
{k: v for k, v in sorted(wc_X1.items(), key=lambda item: item[1], reverse=True)[:10]}

{'karen': 8.69223,
 'wrenching': 8.31882,
 'carpenter': 7.468960000000001,
 'laughter': 7.467879999999999,
 'liked': 6.864090000000003,
 'mom': 6.791519999999999,
 'gut': 6.759419999999999,
 'love': 6.551409999999997,
 'camp': 6.533080000000001,
 'hr': 6.1393699999999995}

In [158]:
{k: v for k, v in sorted(wc_X2.items(), key=lambda item: item[1], reverse=True)[:10]}

{'hopper': 8.372459999999998,
 'jake': 7.63837,
 'movie': 7.267059999999995,
 'film': 6.936379999999998,
 'shakespeare': 5.99276,
 'oddness': 5.53033,
 'terrible': 4.943440000000001,
 'parent': 4.751790000000001,
 'actor': 4.672620000000001,
 'bad': 4.430020000000002}